In [1]:
import gym
import numpy as np
import pandas as pd
env = gym.make('FrozenLake-v0')
pd.set_option("display.precision", 3)

### Comparing performance of a Markov process to non-deterministic Monte Carlo solver for a deterministic game

For simple games, where the outcome probabilities of each move are known, and where the game is not so large that computing power makes a markov process impractical, the Markov method should yield the best outcome when training an agent. In this notebook, though, my motivation is to see if using a Monte Carlo process can yield similarly powerful results, so we can understand the potential power of the Monte Carlo method when applied to more complex, non-deterministic problems

----
#### The Game - Frozen Lake

Frozen lake is a game, where a bot has to get from one corner of the lake to another without falling down a hole. https://gym.openai.com/envs/FrozenLake-v0/

The bot "decides" which way to go, but there is also randomness in this - i.e. the computer sometimes makes a different decision for us

We need to train the bot to make the right decisions. Here is what the 'board' looks like:

S = Start

F = Frozen (can walk on)

H = Hole (game ends)

G = Goal (game ends and you get a reward)

In [48]:
env.render()


SFFF
FHFH
FFFH
HFFG


--- 
This is a deterministic game. Under the hood, we can see the probabilities associated with any 'move' at a certain square. For example, here, in state 10

- if we press 0(left), we have a 1/3 chance of going left, a 1/3 chance of going up, and a 1/3 chance of going right.

- The second column refers to the step we would be in

- the third column refers to the reward associated with this move

- the fourth column refers to whether this would end the game

- Each 'direction' only has three options in this game, so for one of the directions, the chance is 0. For example, if we press 0 (left), we cannot reach square 7(right, in this case)

In [211]:
env.P[10]

{0: [(0.3333333333333333, 2, 0.0, False),
  (0.3333333333333333, 5, 0.0, True),
  (0.3333333333333333, 10, 0.0, False)],
 1: [(0.3333333333333333, 5, 0.0, True),
  (0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 7, 0.0, True)],
 2: [(0.3333333333333333, 10, 0.0, False),
  (0.3333333333333333, 7, 0.0, True),
  (0.3333333333333333, 2, 0.0, False)],
 3: [(0.3333333333333333, 7, 0.0, True),
  (0.3333333333333333, 2, 0.0, False),
  (0.3333333333333333, 5, 0.0, True)]}

---

#### Treating it as a markov decision problem

For deterministic games, a Markov decision process will work, assuming there are not too many combinations and state-action pairs for the computer to handle. This is quite a simple game, so there won't be.

Markov generally works by calculating the 'value' of any state action by making it equal to the immediate reward + the value of the state it leads to you to move to. By recursively calculating this for all states, we gradually converge to a stable state-action value matrix.

We will approach this is a policy model, which has three 'steps':

1) Define a function to evaluate the value of each state according to a certain policy

2) for a given state, determine the value of each action

3) iterate on the policy, by changing the 'best' action in the policy if another action has a higher value.

Here we start with all the states being zero, and then iteratively updating the value of each square, and then using the value fucntion, which incorporates the probability of being in a future state and that state's value.

In [198]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    # Create a counter which we return later to show how long it took
    evaluation_iterations = 1
    # Initialize a value function for each state as zero
    V = np.zeros(environment.nS) #V is value matrix, v is value of a state
    
    '''goal is to keep iterating until the change in value is very very small (below a threshold)'''

    for i in range(int(max_iterations)):
        # Initialize a change of value function as zero
        delta = 0
        # Iterate though each state
        for state in range(environment.nS):
           # Initialize a new value of current state
            v = 0
            '''For each action and its probability in a given state'''
            for action, action_probability in enumerate(policy[state]):
                '''look at the possible next states of these actions, the prob of that, the reward of that whether it lead to termination'''
                for state_probability, next_state, reward, terminated in environment.P[state][action]:
                    # '''calculate expected value which is the chance of this value happening * reward and the value in the matrix currently of the next state'''
                    v += action_probability * state_probability * (reward + discount_factor * V[next_state]) #on iteration one, only current reward is considered

           # Calculate the absolute change of value function
            delta = max(delta, np.abs(V[state] - v)) #at beginning of iteration thorugh the whole thing, delta set to 0. so this will capture the highest state delta
           # Update value function
            V[state] = v
        evaluation_iterations += 1

        # Terminate if value change is insignificant
        if delta < theta:
            print(f'Policy evaluated in {evaluation_iterations} iterations.')
            return V

In [199]:
def one_step_lookahead(environment, state, V, discount_factor):
    '''this is a function fo a given state, what is the value of each action, so returns a vector of 4 values (if up down left right)'''
    action_values = np.zeros(environment.nA) #nA = (4,1)
    for action in range(environment.nA):
        for probability, next_state, reward, terminated in environment.P[state][action]:
            action_values[action] += probability * (reward + discount_factor * V[next_state])
    return action_values

In [200]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    # Start with a random policy
    #this is a state x action matrix
    policy = np.ones([environment.nS, environment.nA]) / environment.nA
    # Initialize counter of evaluated policies
    evaluated_policies = 1
    # Repeat until convergence or critical number of iterations reached
    for i in range(int(max_iterations)):
        stable_policy = True
        # Evaluate current policy = create a V matrix of value of any state.
        V = policy_evaluation(policy, environment, discount_factor=discount_factor)
        # Go through each state and try to improve actions that were taken (policy Improvement)
        for state in range(environment.nS):
            # '''Pick the action in this state which currently has the highest probability of occuring, or "current action" '''
            current_action = np.argmax(policy[state])
            # Look one step ahead and evaluate if current action is optimal
            # We will try every possible action in a current state
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select a better action
            best_action = np.argmax(action_value)
            # If action didn't change
            if current_action != best_action:
                stable_policy = False
                # Greedy policy update
                policy[state] = np.eye(environment.nA)[best_action] #creates a policy of one action per state.
        evaluated_policies += 1
        # If the algorithm converged and policy is not changing anymore, then return final policy and value function
        if stable_policy:
            print(f'Evaluated {evaluated_policies} policies.')
            return policy, V

In [203]:
def play_episodes(environment, n_episodes, policy):
    wins = 0
    total_reward = 0
    for episode in range(n_episodes):
        terminated = False
        state = environment.reset()
        while not terminated:
            # Select best action to perform in a current state
            action = np.argmax(policy[state])
            # Perform an action an observe how environment acted in response
            next_state, reward, terminated, info = environment.step(action)
            # Summarize total reward
            total_reward += reward
            # Update current state
            state = next_state
            # Calculate number of wins over episodes
            if terminated and reward == 1.0:
                wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

# Number of episodes to play
n_episodes = 10000
# Functions to find best policy
solvers = [('Policy Iteration', policy_iteration)] #[('Value Iteration', value_iteration),
           
for iteration_name, iteration_func in solvers:
    # Load a Frozen Lake environment
    environment = gym.make('FrozenLake-v0')
    # Search for an optimal policy using policy iteration
    policy, V = iteration_func(environment.env)
    # Apply best policy to the real environment
    wins, total_reward, average_reward = play_episodes(environment, n_episodes, policy)
    print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
    print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward} \n\n')

Policy evaluated in 66 iterations.
Policy evaluated in 170 iterations.
Policy evaluated in 428 iterations.
Evaluated 4 policies.
Policy Iteration :: number of wins over 10000 episodes = 7386
Policy Iteration :: average reward over 10000 episodes = 0.7386 




#### Monte Carlo Approach
---
Monte Carlo works by allowing the agent to 'explore', and then assigning any reward for the game just played to all the actions played in the game. For example, if we play a game and we get a reward of 1, and during the game we played "left" from square 2, then that state-action pair will be assigned a value of 1. Over time, actions that have been assigned high value in previous games will be played more frequently, although the rate at which we continue exploring low value options is a parameter which, when increased, can increase our chances of finding the optimal solution, but at the cost of poor performance in the train set.

In [203]:
#Note: This is the dictionary of action indexes
act_dict = {0:'left',1:'down',2:'right',3:'up'}

---
Quick Example of playing the game

In [239]:
#An example of playing the game
env.reset()
done = False
while done == False:
    action = np.random.choice(array_of_choices,p=random_policy_matrix[state])
    state, reward, done, info = env.step(action)
    print(state,action)
    env.render()

4 1
  (Down)
SFFF
FHFH
FFFH
HFFG
8 1
  (Down)
SFFF
FHFH
FFFH
HFFG
12 0
  (Left)
SFFF
FHFH
FFFH
HFFG


---
Creating Core functions for the game

In [168]:
'''function to initialise the state'''

def init_env():
    sa_occurence_matrix = np.zeros((16,4)) #needs to be reinitialized every time.
    env.reset()
    done = False
    state=0
    rew= 0
    return sa_occurence_matrix, env, done, state, rew

'''Function that plays the game once, recording the states and actions and assigning reward as appropriate.'''

def one_iteration(sa_valmat,policy='random',epsilon=0.1):
    '''sa_valmat is a 16x4x2 matrix, (16 states, 4 actions, and a channel for count and value)'''
    sa_occurence_matrix, env, done, state, rew = init_env()
    step_count = 0
    while done == False:
        step_count +=1
        if policy == 'random':
            action = np.random.choice(array_of_choices,p=random_policy_matrix[state])
        elif policy == 'tiny_epsilon':
            action = tiny_epsilon_choice(sa_valmat,state, random_policy_matrix,epsilon = epsilon)
        elif policy == 'softmax':
            action = np.random.choice(array_of_choices,p=np.exp(sa_valmat[state,:,0])**0.3/np.sum(np.exp(sa_valmat[state,:,0])**0.3))
        sa_occurence_matrix[state,action] += 1 #could be plus equal one if we're using each time.
        state, reward, done, info = env.step(action)
        rew = max(rew,reward)
    for i in range(sa_valmat.shape[0]):
        for j in range(sa_valmat.shape[1]):
            if sa_occurence_matrix[i,j] >= 1:
                #increment the count channel
                sa_valmat[i,j,1] += sa_occurence_matrix[i,j]
                #get new value average column - here we allocate a point every time we took an action in a state - so if we went left three times from 4, that would be incremented by 4.
                sa_valmat[i,j,0] += (sa_occurence_matrix[i,j]/ sa_valmat[i,j,1]) * (rew - sa_valmat[i,j,0])
    
    return sa_valmat, rew, step_count

'''Helper function for when we use tiny epsilon as an exploration method'''

def tiny_epsilon_choice(sa_valmat, state, random_policy_matrix, epsilon,rubber='no'):
    '''select the best option most of the time'''
    random_shot = np.random.uniform(0,1)
    if random_shot >= epsilon:
        action = np.argmax(sa_valmat[state,:,0])
    else:
        action = np.random.choice(array_of_choices,p=random_policy_matrix[state])
    return action



#### Let's train and test this for the three different epsilon policies

- Random: allowing it to at randomly pick a route, max exploration

- Softmax - using a softmax function with a dampener of 0.3 (so it's slightly more exploratory)

- Using Tiny Epsilon, with a value of 0.2 to pick the random choice sometimes and the best choice most of the time.

In [212]:
def train_and_test(policy,train_epochs,test_epochs):
    
    '''Create empty state action value matrix, random policy matrix and the choices available to us'''
    
    sa_valmat = np.zeros((16,4,2)) #we'll update it using v(sa) = v(sa) 1/N(sa) + [r(t) - v(sa)]
    sa_valmat[:,:,0] +=0.01 #create tiny bit of value for all SA's in order for divisors to work
    random_policy_matrix = np.zeros((16,4)) + 0.25
    array_of_choices = np.array([0,1,2,3])
    
    #We'll counter train and test wins over the course of the function
    train_wins =0
    test_wins = 0
    
    
    '''we're going to initialise the game 2000 times, and randomly walk throughout the game and then assign an award to each s/a that occurs.'''     
    for i in range(10):
        sa_valmat, rew, _ = one_iteration(sa_valmat,policy='random')
        train_wins += rew
    
    '''Then were going to train using tiny epsilon, random or softmax'''
    for i in range(train_epochs):
        #used when we need it, converges to about 0.05 after high number of trials
        epsilon = 0.5 - (np.exp(i/train_epochs)/(1+np.exp(i/train_epochs)))/2
        sa_valmat, rew, step_count = one_iteration(sa_valmat,policy=policy,epsilon=epsilon)
        train_wins+=rew
    
    '''Then were going to test by setting epsilon to zero'''
    for i in range(test_epochs):
        epsilon = 0
        sa_valmat, rew_pol, step_count = one_iteration(sa_valmat,policy='tiny_epsilon',epsilon=epsilon) #here we use tiny epsilon with epsilon equals 0
        wins += rew_pol
        logging_test_step_count+= step_count
        
    return test_win, test_wins/test_epochs, train_wins, train_wins/train_epochs


In [182]:
softmax_figures = train_and_test('softmax', 500000, 5000)
tiny_ep_figures = train_and_test('tiny_epsilon', 500000, 5000)
random_pol_figures = train_and_test('random', 500000, 5000)

3835497
9004453
3838047


In [197]:
summary_table = pd.DataFrame(data=[softmax_figures,tiny_ep_figures,random_pol_figures],columns=['Test wins','test acc','train wins','train acc']).T.rename(columns={0:'softmax',1:'tiny epsilon',2:'random policy'})
display(summary_table)

,softmax,tiny epsilon,random policy
Test wins,3613.000,2499.000,3645.000
test acc,0.723,0.500,0.729
train wins,7199.000,100993.000,6948.000
train acc,0.014,0.202,0.014


#### Summary

Softmax and Random policy had a lower train accuracy, as they tend to favour exploration over exploitation.

tiny epsilon recorded significantly higher train wins; however, due to its favouring of what it believed to be the best solutions, it failed to explore enough, and there doesn't have the accuracy in the test that the other algos created.

finally, it's clear that we achieved the best results using a deterministic approach, which makes sense, because if this approach is available to us, it will normally yield the best answer, quickest.

However, we have shown that using a non-deterministic approach, we can achieve the same results as the Markov approach. This will bode well if we need to apply it to a non-deterministic problem.